In [1]:
"""Display settings"""
from IPython.display import HTML, display


"""MySQL connection related functions and variables"""

import pymysql

def open_conn():
    """open the connection before each test case"""
    conn = pymysql.connect(user='public', password='ece656yelp',
                                   host='maindb.czbva1am4d4u.us-east-2.rds.amazonaws.com',
                                   database='yelp_db')
    return conn

def close_conn(conn):
    """close the connection after each test case"""
    conn.close()

def executeQuery(conn, query, commit=False, returnResult=True):
    """ fetch result after query"""
    cursor = conn.cursor()
    query_num = query.count(";")
    if query_num > 1:
        if not returnResult:
            cursor.execute(query)
            return
        else:
            for result in cursor.execute(query, params=None, multi=True):
                if result.with_rows:
                    result = result.fetchall()
    else:
        cursor.execute(query)
        result = cursor.fetchall()
    # we commit the results only if we want the updates to the database
    # to persist.
    if commit:
        conn.commit()
    else:
        conn.rollback()
    # close the cursor used to execute the query
    cursor.close()
    return result

yelp_conn = open_conn()

In [6]:
query_1 = "SELECT id, date FROM review WHERE unix_timestamp(date) <= unix_timestamp('2004-10-01')\
           OR unix_timestamp(date) >= unix_timestamp('2018-01-01');"

%time result_1 = executeQuery(yelp_conn, query_1)

#result_1: "15.292s, returns [('03B9-gqbeGoMmPJbNzNT5w', '2004-09-15 00:00:00'),('PbIY2aIyszb6he6J-ey67w', '2004-07-22 00:00:00'
#)]"

CPU times: user 3.07 ms, sys: 0 ns, total: 3.07 ms
Wall time: 2.58 s


In [8]:
query_2 =  "SELECT user.id FROM (user INNER JOIN review ON user.id = review.user_id)\
            WHERE user.yelping_since > review.date GROUP BY user.id;"

%time result_2 = executeQuery(yelp_conn, query_2)

#result_2: 'Too long, >5min'

CPU times: user 4.61 ms, sys: 3.2 ms, total: 7.8 ms
Wall time: 11.1 s


In [2]:
query_3 =  "SELECT user.id FROM (user INNER JOIN elite_years ON user.id = elite_years.user_id)\
            WHERE YEAR(user.yelping_since) > elite_years.year GROUP BY user.id;"
%time result_3 = executeQuery(yelp_conn, query_3)

#result_3: 'Too long, >5min'

CPU times: user 2.08 ms, sys: 0 ns, total: 2.08 ms
Wall time: 262 ms


In [ ]:
query_4 = "SELECT COUNT(*) FROM checkin JOIN (SELECT hours.business_id, SUBSTRING_INDEX(hours, '|', 1)\
           AS day_of_week, SUBSTRING_INDEX(SUBSTRING_INDEX(hours, '|', - 1), '-', 1) AS opening_time,\
           SUBSTRING_INDEX(SUBSTRING_INDEX(hours, '|', - 1), '-', - 1) AS closing_time FROM hours)\
           AS a ON a.business_id = checkin.business_id\
           AND a.day_of_week = SUBSTRING_INDEX(checkin.date, '-', 1)\
           WHERE a.opening_time > SUBSTRING_INDEX(checkin.date, '-', - 1)\
           AND a.closing_time < SUBSTRING_INDEX(checkin.date, '-', - 1);"
%time result_4 = executeQuery(yelp_conn, query_4)

#result_4: 'Too long for both, >5min, need to create and index the new columns instead'

In [3]:
query_5 = "select count(*) from user join (select count(user_id) as countedReviews, user_id from\
           review group by user_id) as a on a.user_id = user.id where a.countedReviews > review_count;"
%time result_5 = executeQuery(yelp_conn, query_5)

#result_5: 'Too long, >5min'

CPU times: user 4.56 ms, sys: 687 µs, total: 5.24 ms
Wall time: 1min 3s


In [5]:
query_6 = "select count(*) from elite_years join \
           (SELECT user_id, SUBSTRING_INDEX(date, '-', 1) AS year FROM review) \
           as a on a.user_id=elite_years.user_id and a.year = elite_years.year \
           group by elite_years.user_id, elite_years.year;"
%time result_6 = executeQuery(yelp_conn, query_6)

#result_6: 'Too long, >5min'

In [ ]:
query_7 = "create table hours_new(\
	id int(11) not null auto_increment, primary key (id)\
    ,business_id varchar(255)\
    ,day_of_week varchar(9)\
    ,opening_time time\
    ,closing_time time\
    );\
    drop procedure if exists dowhile_hours;\
    delimiter //\
    create procedure dowhile_hours ()\
    begin declare v1 int default 821044;\
    while v1 > 0 do\
    insert into hours_new(id) values (null);\
    set v1 = v1 - 1;\
    end while;\
    end//\
    delimiter ;\
    call dowhile_hours();\
    update hours_new set day_of_week = (\
        select SUBSTRING_INDEX(hours, '|', 1) from hours where hours_new.id = hours.id);\
    update hours_new set opening_time = (\
        select SUBSTRING_INDEX(SUBSTRING_INDEX(hours, '|', - 1), '-', 1) from hours where hours_new.id = hours.id);\
    update hours_new set closing_time = (\
        select SUBSTRING_INDEX(SUBSTRING_INDEX(hours, '|', - 1), '-', - 1) from hours where hours_new.id = hours.id);\
    update hours_new set business_id = (\
        select business_id from hours where hours_new.id = hours.id);"
![hours_new_with_index](../../pythonAnalysis/figures/hours_new_with_index.png)

In [ ]:
query_8 = "create index idx_day_of_week on hours_new(day_of_week);\
create index idx_opening_time on hours_new(opening_time);\
create index idx_closing_time on hours_new(closing_time);"
![hours_new_with_index](../../pythonAnalysis/figures/hours_new_indexes_with_index.png)

In [ ]:
query_9 = "create table checkin_new(\
	id int(11) not null auto_increment, primary key (id)\
    ,business_id varchar(255)\
    ,day_of_week varchar(9)\
    ,checkin_time time\
    );\
    drop procedure if exists dowhile_checkin;\
    delimiter //\
    create procedure dowhile_hours ()\
    begin declare v1 int default 3891600;\
    while v1 > 0 do\
    insert into checkin_new(id) values (null);\
    set v1 = v1 - 1;\
    end while;\
    end//\
    delimiter ;\
    call dowhile_checkin();\
    update checkin_new set day_of_week = (\
        select SUBSTRING_INDEX(date, '-', 1) from checkin where checkin_new.id = checkin.id);\
    update checkin_new set checkin_time = (\
        select SUBSTRING_INDEX(date, '-', -1) from checkin where checkin_new.id = checkin.id);\
    update checkin_new set business_id = (\
        select business_id from checkin where checkin_new.id = checkin.id);"
![hours_new_with_index](../../pythonAnalysis/figures/checkin_new_with_index.png)

In [ ]:
query_8 = "create index idx_day_of_week on checkin_new(day_of_week);\
create index idx_opening_time on checkin_new(checkin_time);"
![hours_new_with_index](../../pythonAnalysis/figures/checkin_new_indexes_with_index.png)